In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#model
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error

import sklearn.linear_model as LM
from sklearn import svm
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from pytorch_tabnet.tab_model import TabNetRegressor
from pytorch_tabnet.augmentations import RegressionSMOTE
from lce import LCERegressor

import torch
from torch import nn
from torch.utils import data

import os
import re

import pickle

plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rc('axes', unicode_minus=False)

In [ ]:
# functions

def splitlabel(data,input,output):
    X = data[:,input]
    y = data[:,output]
    return X,y

def average_choose_data(data_x,data_y,dividearr,divide_n,aver_num,seed=None):
    datanew_x,datanew_y = np.zeros((0,data_x.shape[1])),np.zeros(0)
    for ii in range(divide_n):
        low,big = dividearr[ii],dividearr[ii+1]
        cond = np.where(np.logical_and(data_y>=low,data_y<=big))[0]
        tmpx,tmpy = data_x[cond,:],data_y[cond]
        if seed != None:
            np.random.seed(seed)
        verifyidx = np.random.choice(range(tmpy.shape[0]), size=aver_num, replace=False)
        datanew_x = np.vstack((datanew_x,tmpx[verifyidx,:]))
        datanew_y = np.hstack((datanew_y,tmpy[verifyidx]))

    return datanew_x,datanew_y

def raw2process(datax): # make sure the features should have the column: *,7v,*,*,*,19v,*,*,37h,37v
    datanew = np.zeros((datax.shape[0],6))
    datanew[:,0] = datax[:,1] # 7v
    datanew[:,1] = datax[:,5] # 19V
    datanew[:,2] = datax[:,-1] # 37v
    datanew[:,3] = (datax[:,-1]-datax[:,5])/(datax[:,-1]+datax[:,5]) #gr 37v/19v
    datanew[:,4] = (datax[:,5]-datax[:,1])/(datax[:,5]+datax[:,1]) #gr 19v/7v
    datanew[:,5] = (datax[:,-1]-datax[:,-2])/(datax[:,-1]+datax[:,-2]) #pr37
    return datanew

def normal(data_xraw,data_yraw):
    data_x = np.zeros_like(data_xraw)
    minmax = np.zeros((2,data_x.shape[1]+1))
    for jj in range(data_x.shape[1]):
    # for jj in range(3):
        minmax[0,jj],minmax[1,jj] = np.min(data_xraw[:,jj]),np.max(data_xraw[:,jj])
        data_x[:,jj] = 2*(data_xraw[:,jj]-minmax[0,jj])/(minmax[1,jj]-minmax[0,jj])-1

    minmax[0,-1],minmax[1,-1] = np.min(data_yraw),np.max(data_yraw)
    data_y = 2*(data_yraw-minmax[0,-1])/(minmax[1,-1]-minmax[0,-1])-1

    return data_x, data_y, minmax

def data_MSE(model,X_train,X_test,y_train,y_test):
    y_train_pred = model.predict(X_train)
    MSEtrain,R2train = mean_squared_error(y_train_pred,y_train),r2_score(y_train_pred,y_train)
    MAEtrain = mean_absolute_error(y_train_pred,y_train)
    biastrain = np.mean(y_train_pred-y_train)
    print('train-MSE ',MSEtrain,' r2 ',R2train,'MAE',MAEtrain,'bias',biastrain)
    y_test_pred = model.predict(X_test)
    MSEtest,R2test = mean_squared_error(y_test_pred,y_test),r2_score(y_test_pred,y_test)
    MAEtest = mean_absolute_error(y_test_pred,y_test)
    biastest = np.mean(y_test_pred-y_test)
    print('test-MSE ',MSEtest,' r2 ',R2test,'MAE',MAEtest,'bias',biastest)
    return [np.sqrt(MSEtest),R2test,MAEtest,biastest]

def keep_digits(s):
    return re.sub(r'\D', '', s)

def get_all_files(folder):
    all_files = []
    for root, dirs, files in os.walk(folder):
        for file in files:
            all_files.append(keep_digits(file))
    return all_files

def normal_out_func(pre,normal_out):
    return (pre+1)/2*(normal_out[1]-normal_out[0])+normal_out[0]

def get_predict(model,datan,normalize_out):
    pre_1 = model.predict(datan)
    pre = normal_out_func(pre_1,normalize_out)
    pre[np.where(pre<0)[0]]=np.nan
    return pre.reshape((-1,1))

def process_predict(data_s,normalize_in,normalize_out,models):
    data_s2 = 2*(data_s-normalize_in[0,:])/(normalize_in[1,:]-normalize_in[0,:])-1

    model_knn,model_et,model_tabnet = models

    pre_knn = get_predict(model_knn,data_s2,normalize_out)
    pre_et = get_predict(model_et,data_s2,normalize_out)
    pre_tabnet = get_predict(model_tabnet,data_s2,normalize_out)

    return np.hstack((pre_knn,pre_et,pre_tabnet))

In [ ]:
# import train data

# pre-process data

'''
There should be the column like ['lon','lat',...(10 features told in README.md)],
or you need to modify the function **splitlabel** and **raw2process**.
'''

data0 = np.array(pd.read_csv(None))

data_x_raw,data_y_raw = splitlabel(data0,range(2,12),1)

data_x_p1,data_y_p1 = average_choose_data(data_x_raw,data_y_raw,
                                              [ 0.056, 0.123 , 0.1896, 0.2561, 0.3895],
                                              4,36,42) # clean data

data_x_p1 = raw2process(data_x_p1) # combine the features

data_x,data_y,minmax = normal(data_x_p1,data_y_p1) # normalization


In [ ]:
# build all models (13 in total)

# X_train,X_test,y_train,y_test = train_test_split(data_x,data_y ,test_size = 0.2)

X_train = data_x; y_train = data_y

# 0--Multi-Linear Regression
model0=LM.LinearRegression()
model0.fit(X_train,y_train)

# 1--Support Vector Machine
C=30;E=0.06;G=0.4
model1=svm.SVR(C=C,epsilon=E,gamma=G)
model1.fit(X_train,y_train)

# 2--XGBoost
model2=XGBRegressor()
model2.fit(X_train,y_train)

# 3(1)--KNN-Average
model3=KNeighborsRegressor(n_neighbors=5,weights='uniform')
model3.fit(X_train,y_train)

# 3(2)--KNN-Euclidean Distance
model3_2 = KNeighborsRegressor(n_neighbors=5,weights='distance')
model3_2.fit(X_train,y_train)

# 4--Gradient Boosting
model4 = GradientBoostingRegressor(learning_rate=0.01,n_estimators=100)
model4.fit(X_train,y_train)

# 5--Random Forest
model5 = RandomForestRegressor(n_estimators=100)
model5.fit(X_train, y_train)

# 6--LightGBM
model6 = lgb.LGBMRegressor(metric='rmse')
model6.fit(X_train, y_train)

# 7--Catboost
model7 = CatBoostRegressor(
    iterations=1000,  # iteration
    learning_rate=0.1,  # lr
    depth=7,  # depth
    loss_function='RMSE',  # loss function
    verbose=100
)
model7.fit(X_train, y_train)


# 8--Extra Trees
model8 = ExtraTreesRegressor(n_estimators=100)
model8.fit(X_train, y_train)

# 9--AdaBoost
model9 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4), n_estimators=500)
model9.fit(X_train, y_train)

# 10--TabNet
model10 = TabNetRegressor()
model10.fit(X_train, y_train.reshape(-1, 1),
           max_epochs=100,
        patience=20,
        batch_size=12, virtual_batch_size=12,
        num_workers=0,
        drop_last=False,
        augmentations=RegressionSMOTE(p=0.2), #aug
           )

# 11--LCE
model11 = LCERegressor()
model11.fit(X_train, y_train)

# 12--NN
# 13--resnet
def np2torch(arr):
    arr1 = torch.from_numpy(arr)
    arr1 = arr1.float()
    return arr1
def torch2np(arr):
    return arr.detach().numpy()
features, labels = np2torch(X_train),np2torch(np.reshape(y_train,(-1,1)))

def load_array(data_arrays,batch_size,is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=is_train)

batch_size = 12
data_iter = load_array((features,labels),batch_size)

net1 = nn.Sequential(
    nn.Linear(6,64),nn.ReLU(),nn.Dropout(0.25),
    nn.Linear(64,128),nn.ReLU(),nn.Dropout(0.25),
    nn.Linear(128,64),nn.ReLU(),
    nn.Linear(64,1)
)

class ResNet(nn.Module):
    def __init__(self,in_features_dim,out_features_dim):
        super(ResNet,self).__init__()
        self.net2_1 = nn.Sequential(nn.Linear(in_features_dim,64),nn.ReLU(),nn.Dropout(0.25))
        self.net2_2 = nn.Sequential(nn.Linear(64,256),nn.ReLU(),nn.Dropout(0.25),
                               nn.Linear(256,256),nn.ReLU(),nn.Dropout(0.25),
                               nn.Linear(256,64),nn.ReLU())
        self.net2_3 = nn.Sequential(nn.Linear(64,out_features_dim))

    def forward(self,x):
        x1 = self.net2_1(x)
        residual = x1
        x2 = self.net2_2(x1)
        x2 += residual
        return self.net2_3(x2)

net2 = ResNet(6,1)

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std = 0.01)
net1.apply(init_weights)
net2.apply(init_weights)

loss = nn.MSELoss()
trainer1 = torch.optim.Adam(net1.parameters(),lr=0.0005)
trainer2 = torch.optim.Adam([{
        "params" :net2[0].weight,
        'weight_decay':1},{
            "params":net2[0].bias
        }
    ],lr = 0.05)

num_epochs = 1000

for epoch in range(num_epochs):
    net1 = net1.train()
    for X,y in data_iter:
        trainer1.zero_grad()
        l = loss(net1(X),y)
        l.backward()
        trainer1.step()
    train_l = loss(net1(features),labels)
    if (epoch+1) % 100==0:
        print(f'epoch {epoch+1}, loss {train_l:f}')

for epoch in range(num_epochs):
    net2 = net2.train()
    for X,y in data_iter:
        trainer2.zero_grad()
        l = loss(net2(X),y)
        l.backward()
        trainer2.step()
    train_l = loss(net2(features),labels)
    if (epoch+1) % 100==0:
        print(f'epoch {epoch+1}, loss {train_l:f}')

class Network():
    def __init__(self,net):
        self.net = net
        self.net = self.net.eval()
    def predict(self,X):
        Xtorch = np2torch(X)
        Ytorch = self.net(Xtorch)
        return torch2np(Ytorch).reshape((-1))

model12 = Network(net1)
model13 = Network(net2)

models = [model0,model1,model2,model3,model3_2,model4,model5,model6,model7,model8,model9,model10,model11,model12,model13]
Names = ['linear','svm','xgb','knn-uni','knn-dis','gb','rf','lgbm','cb','et','adb','tab','lce','nn','res']


In [ ]:
# process the whole data

# model_knn = KNeighborsRegressor(n_neighbors=5,weights='uniform')
# model_knn.fit(X_train,y_train)
#
# model_et = ExtraTreesRegressor(n_estimators=100)
# model_et.fit(X_train, y_train)
#
# model_tabnet = TabNetRegressor()
# model_tabnet.fit(X_train, y_train.reshape(-1, 1),
#            max_epochs=100,
#         patience=20,
#         batch_size=12, virtual_batch_size=12,
#         num_workers=0,
#         drop_last=False,
#         augmentations=RegressionSMOTE(p=0.2), #aug
#            )

model_knn = model3
model_et = model8
model_tabnet = model10

with open('knn.pkl', 'wb') as file:
    pickle.dump(model_knn, file)
with open('et.pkl', 'wb') as file:
    pickle.dump(model_et, file)
with open('tabnet.pkl', 'wb') as file:
    pickle.dump(model_tabnet, file)

df = pd.DataFrame(minmax)
df.to_csv('minmax.csv')

# import outside data

directory_urf = 'data/'
# with open('knn.pkl', 'rb') as file:
#     model_knn = pickle.load(file)
# with open('et.pkll', 'rb') as file:
#     model_et = pickle.load(file)
# with open('tabnet.pkl', 'rb') as file:
#     model_tabnet = pickle.load(file)
# df = pd.read_csv('minmax.csv')
# minmax = np.array(df)

filein = get_all_files(directory_urf)
normalize_in = minmax[:,:-1]
normalize_out = minmax[:,-1]

models_final = [model_knn,model_et,model_tabnet]
modelfiles = ['KNN','ExtraTrees','TabNet']

save_urf = 'result/'

for file in filein:
    datain = pd.read_csv(directory_urf+file+'.csv')
    datain = np.array(datain)
    datain_x = datain[:,-10:]
    datain_x = raw2process(datain_x)
    dataout_y = np.zeros((datain_x.shape[0],3))

    tmp_not_nan_idx = []

    for tt in range(datain_x.shape[0]):
        data_line = datain_x[tt,:]
        if np.isnan(data_line).any():
            dataout_y[tt,:] = np.nan
        else:
            tmp_not_nan_idx.append(tt)
    print(file+' Start predicting!')
    dataout_y[tmp_not_nan_idx,:] = process_predict(datain_x[tmp_not_nan_idx,:],normalize_in,normalize_out,models_final)

    dataout = np.hstack((datain[:,:2],dataout_y))

    print(file,' ',dataout.shape)
    # save as CSV
    for tt in range(3):
        df = pd.DataFrame(dataout[:,tt+2].reshape((-1,1)))
        df.to_csv(save_urf+modelfiles[tt]+'/'+file+'.csv', index=False)
    print('------------------------------------------')

datain = pd.read_csv(save_urf+filein[0]+'.csv')
datain = np.array(datain)
df = pd.DataFrame(datain[:,:2])
df.to_csv(save_urf+'lonlat.csv', index=False)